In [ ]:
#test

In [1]:
# testing the mean square sgd 
import numpy as np 
from implementations import *

y = np.array([[0.1],
       [0.3],
       [0.5]])
tx = np.array([[2.3, 3.2],
       [1. , 0.1],
       [1.4, 2.3]])
initial_w = np.array([[0.5],
       [1. ]])

MAX_ITERS = 2
GAMMA = 0.1


In [2]:
w, loss = mean_squared_error_gd(y, tx, initial_w, MAX_ITERS, GAMMA)
print(w) 
print(loss)

Gradient Descent(0/1): loss=4.067083333333334
Gradient Descent(1/1): loss=0.24939051708333312
[[-0.0505865]
 [ 0.203718 ]]
0.24939051708333312


In [3]:
# testing mean square sgd
y = np.array([[0.1],
       [0.3],
       [0.5]])
tx = np.array([[2.3, 3.2],
       [1. , 0.1],
       [1.4, 2.3]])
initial_w = np.array([[0.5],
       [1. ]])

In [4]:
w, loss = mean_squared_error_sgd(y[:1], tx[:1], initial_w, MAX_ITERS, GAMMA)
print(w)
print(loss)

[[0.0630575]
 [0.39208  ]]
2.761837531250002
